# SOLUTION KEY

In [6]:
import pennylane as qml
import numpy as np
from print_latex import print_state_vector

## NOTEBOOK 1 : Build a one-qubit simulator

In [ ]:
#EXERCISE 1
def qubit_state(alpha, beta):
    """
    Creates an arbitrary qubit state |ψ⟩ according to coefficients alpha and beta.
    
    Args:
        alpha (complex): Amplitude of state |0⟩
        beta (complex): Amplitude of state |1⟩
        
    Returns:
       numpy.ndarray: qubit state vector [alpha, beta]
    """
    # Define |0⟩ and |1⟩
    ket_0 = np.array([1,0])
    ket_1 = np.array([0,1])
    
    # Construct the qubit state |ψ⟩
    psi = (alpha * ket_0) + (beta * ket_1)

    # Retourner the qubit state  
    return psi

# Test with alpha = 1/2 and beta = sqrt(3)/2 * i
alpha_test = 1/2.0
beta_test = np.sqrt(3)/2 * 1j

psi = qubit_state(alpha_test, beta_test)
print("State |ψ⟩ :", psi)

In [ ]:
def is_normalized(state):  
    """Checks that an arbitary quantum state is normalized .  

    Args:  
        state (np.array): qubit state vector

    Returns:  
        Bool: True if the vector is normalized, false otherwise.  
    """  
    norm = np.conj(alpha)*alpha + np.conj(beta)*beta
    return np.isclose(norm, 1)  

# Test the function  
alpha = 1/2  
beta = np.sqrt(3)/2 * 1j  
psi = qubit_state(alpha, beta)  
is_normalized = is_normalized(psi)

print("Qubit state :", psi)  
print("Normalized :", is_normalized)  

In [ ]:
#EXERCISE 3
def apply_H(state, H):
    """
    Applies a quantum gate to a given quantum state. 
    
    Args:
        state (np.array): qubit state vector
        H (np.array): 2x2 unitary matrix representing the Hadmard gate 
        
    Returns:
        np.array: New qubit state after applying the matrix H 
    """
    #  Apply matrix H to the qubit state and return the new state
    return np.dot(H, ket_0)
    
# Define the Hadamard matrix
H = 1/np.sqrt(2) * np.array([[1, 1], [1, -1]])

# Test the function with |0⟩ and H
ket_0 = np.array([1, 0])
new_ket = apply_H(ket_0, H)
print("New quantum state after applying H :", new_ket)

In [ ]:
#EXERCISE 4
def measurement_probabilities(state):  
    """
    Calculates the probabilities of measuring |0⟩ and |1⟩ from a given quantum state.  

    Args:  
        state (np.array): Vector representing the state of the qubit.  

    Returns:  
        tuple: Probabilities (P0, P1) corresponding to measurements |0⟩ and |1⟩.  
    """

    P0 = np.conj(state[0])*state[0]
    P1 = np.conj(state[1])*state[1]
    return P0, P1  

# Define the superposition state
psi = (1 / np.sqrt(2)) * np.array([1, 1])  

# Test the function  
P0, P1 = measurement_probabilities(psi)  
print(P0, P1)  

In [ ]:
#EXERCISE 5
def simulate_measurements(probs, num_samples = 1000):  
    """Simulates quantum measurements by sampling the probability distribution of a quantum state.  

    Args:  
        probs (tuple): |0⟩ and |1⟩ measurement probabilities . 
        num_samples (int): Number of measurements ("shots") to simulate.  

    Returns:  
        dict: Number of occurrences of each result (0 or 1). 
    """  
    results = np.random.choice([0, 1], p = probs, size = num_samples) 
    
     #It's nicer to pack everything into a dictionnary that counts 0 and 1 than to look at a list of 0 and 1.
    return dict(zip(*np.unique(results, return_counts=True)))  

# Probabilities obtained previously  
probs = (P0, P1)  

# Test the function  
measurement_results = simulate_measurements(probs)  
print(measurement_results)  

## NOTEBOOK 2 : PennyLane

In [ ]:
#EXERCISE 1
dev = qml.device("default.qubit", wires = 2)
@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires = 0)
    qml.CNOT(wires = [0,1])
    return qml.state()

In [ ]:
#EXERCISE 2

dev = qml.device('default.qubit', wires = 2)
@qml.qnode(dev)
def phi_minus():
 
    qml.PauliX(wires = 0)
    qml.Hadamard(wires = 0)
    qml.CNOT(wires = [0,1])
    return qml.state()

@qml.qnode(dev)
def psi_plus():

  qml.Hadamard(wires = 0)
  qml.CNOT(wires = [0,1])
  qml.PauliX(wires = 0)
  return qml.state()

@qml.qnode(dev)
def psi_minus():

  qml.Hadamard(wires = 0)
  qml.CNOT(wires = [0,1])
  qml.Z(wires = 0)
  qml.X(wires = 1)
  return qml.state()

print_state_vector(phi_minus())
print_state_vector(psi_plus())
print_state_vector(psi_minus())


In [ ]:
#EXERCISE 3a
dev = qml.device('default.qubit', wires=1)
@qml.qnode(dev)
def prepare_with_gate():
  qml.RX(np.pi/3, wires = 0)
  return qml.state()

print_state_vector(prepare_with_gate())

#EXERCISE 3b
state = np.array([np.sqrt(3)/2, -0.5j])

@qml.qnode(dev)
def prepare_with_StatePrep(state):
  qml.StatePrep(state=state, wires=0)
  return qml.state()

print_state_vector(prepare_with_statePrep(state))

In [ ]:
#EXERCISE 4
num_shots = [1,10,100,1000, 5000]
res = []
for shot in num_shots:
    dev = qml.device('default.qubit', wires = 2, shots = shot)
    @qml.qnode(dev)
    def bell_probs():
        bell()
        return qml.probs()
    
    res.append(bell_probs())
print(res)

dev = qml.device('default.qubit', wires = 1)
@qml.qnode(dev)
def circuit():

    qml.Hadamard(wires = 0)
    qml.Z(wires = 0)

    return qml.expval(qml.Z(wires = 0))
    
print(circuit())